In [1]:
import numpy as np
import tensorflow as tf
import re
import time
 

Lines looks like this: `L1045 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ They do not!`
and conversations looks like this `u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L194', 'L195', 'L196', 'L197']`

So we need to process boths and then map the Lines ids with the conversations


In [2]:
lines = open('cornell movie-dialogs corpus/movie_lines.txt', encoding = 'utf-8', errors = 'ignore').read().split('\n')
conversations = open('cornell movie-dialogs corpus/movie_conversations.txt', encoding = 'utf-8', errors = 'ignore').read().split('\n')
 

In [9]:
# The line will be the key of the data
id2line = {}
for line in lines:
    _line = line.split(' +++$+++ ')
    if len(_line) == 5:
        id2line[_line[0]] = _line[4]
 

In [10]:
# Initial cleaning of the conversations and get a list of all the conversations
conversations_ids = []
for conversation in conversations[:-1]:
    _conversation = conversation.split(' +++$+++ ')[-1][1:-1].replace("'", "").replace(" ", "")
    conversations_ids.append(_conversation.split(','))

Now that we have the conversations and the lines we can join them and make 2 lists: questions and answers that are going to be the inputs and the targets of our ANN.

In the example below `['L194', 'L195', 'L196', 'L197']` this conversation has a "question" L194 and it "answer" L195. So the index of the list of questions will math the same index in the answers one.

In [13]:
questions = []
answers = []
for conversation in conversations_ids:
    for i in range(len(conversation) - 1):
        questions.append(id2line[conversation[i]])
        answers.append(id2line[conversation[i+1]])

In [14]:
# Cleaning function
def clean_text(text):
    text = text.lower()
    text = re.sub(r"i'm", "i am", text)
    text = re.sub(r"he's", "he is", text)
    text = re.sub(r"she's", "she is", text)
    text = re.sub(r"that's", "that is", text)
    text = re.sub(r"what's", "what is", text)
    text = re.sub(r"where's", "where is", text)
    text = re.sub(r"\'ll", " will", text)
    text = re.sub(r"\'ve", " have", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"\'d", " would", text)
    text = re.sub(r"won't", "will not", text)
    text = re.sub(r"can't", "cannot", text)
    text = re.sub(r"[-()\"#/@;:<>{}+=~|.?,]", "", text)
    return text

In [15]:
# Cleaning questions and answers
clean_questions = []
for question in questions:
    clean_questions.append(clean_text(question))

clean_answers = []
for answer in answers:
    clean_answers.append(clean_text(answer))
 

We are going to remove the non freqent words of the corpus to optimize the training, for this problem we will use 5% as the threshold (if the word appear less than the 5% will be removed).

So we are creating a dict with the word as the key and the value will be the number of times it has appeared on the corpus.

In [16]:
word2count = {}
for question in clean_questions:
    for word in question.split():
        if word not in word2count:
            word2count[word] = 1
        else:
            word2count[word] += 1
for answer in clean_answers:
    for word in answer.split():
        if word not in word2count:
            word2count[word] = 1
        else:
            word2count[word] += 1

When filtering the words we are creating a mapping between the words and an integer.

In [17]:
threshold_questions = 20
questionswords2int = {}
word_number = 0
for word, count in word2count.items():
    if count >= threshold_questions:
        questionswords2int[word] = word_number
        word_number += 1
threshold_answers = 20
answerswords2int = {}
word_number = 0
for word, count in word2count.items():
    if count >= threshold_answers:
        answerswords2int[word] = word_number
        word_number += 1

Tokenize the words:

- `<PAD>: Padding for empty positions`
- `<EOS>: End of String`
- `<OUT>: Filtered words`
- `<SOS>: Start of String`

In [18]:
tokens = ['<PAD>', '<EOS>', '<OUT>', '<SOS>']

# Gives amn Integer for the token
for token in tokens:
    questionswords2int[token] = len(questionswords2int) + 1
for token in tokens:
    answerswords2int[token] = len(answerswords2int) + 1

# Create an inverse dic to map the words
answersints2word = {w_i: w for w, w_i in answerswords2int.items()}
 
# Adding the End Of String token to the end of every answer
for i in range(len(clean_answers)):
    clean_answers[i] += ' <EOS>'

# Translating all the questions and the answers into integers
# and Replacing all the words that were filtered out by <OUT> 
questions_into_int = []
for question in clean_questions:
    ints = []
    for word in question.split():
        if word not in questionswords2int:
            ints.append(questionswords2int['<OUT>'])
        else:
            ints.append(questionswords2int[word])
    questions_into_int.append(ints)
answers_into_int = []
for answer in clean_answers:
    ints = []
    for word in answer.split():
        if word not in answerswords2int:
            ints.append(answerswords2int['<OUT>'])
        else:
            ints.append(answerswords2int[word])
    answers_into_int.append(ints)
 

To speed ut the training we will wort the questions and the answers by the lenght of the questions.

In [19]:
sorted_clean_questions = []
sorted_clean_answers = []
for length in range(1, 25 + 1):
    for i in enumerate(questions_into_int):
        if len(i[1]) == length:
            sorted_clean_questions.append(questions_into_int[i[0]])
            sorted_clean_answers.append(answers_into_int[i[0]])

In [23]:
np.save("sorted_clean_questions",sorted_clean_questions)
np.save("sorted_clean_answers",sorted_clean_answers)